In [173]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import openpyxl
import nltk
from bs4 import BeautifulSoup
import sklearn


import warnings
warnings.filterwarnings('ignore')

In [174]:
import os
os.listdir("../Datasets")

['1.py', 'amazon_reviews.csv', 'IMDB_Dataset.xlsx']

In [175]:
review_df = pd.read_csv("../Datasets/amazon_reviews.csv")
review_df.head(4)

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4.0,No issues.,2014-07-23,138,0,0,0,0,0.0,0.0
1,1,0mie,5.0,"Purchased this for my device, it worked as adv...",2013-10-25,409,0,0,0,0,0.0,0.0
2,2,1K3,4.0,it works as expected. I should have sprung for...,2012-12-23,715,0,0,0,0,0.0,0.0
3,3,1m2,5.0,This think has worked out great.Had a diff. br...,2013-11-21,382,0,0,0,0,0.0,0.0


In [176]:
review_df.shape

(4915, 12)

In [177]:
review_df.dtypes

Unnamed: 0                int64
reviewerName             object
overall                 float64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object

In [178]:
review_df.describe()


,Unnamed: 0,overall,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
count,4915.000000,4915.000000,4915.000000,4915.000000,4915.000000,4915.000000,4915.000000,4915.000000,4915.000000
mean,2457.000000,4.587589,437.367040,1.311089,0.210376,1.521465,1.100712,0.075468,0.020053
std,1418.982617,0.996845,209.439871,41.619161,4.023296,44.123095,39.367949,0.256062,0.077187
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,-130.000000,0.000000,0.000000
25%,1228.500000,5.000000,281.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2457.000000,5.000000,431.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3685.500000,5.000000,601.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,4914.000000,5.000000,1064.000000,1952.000000,183.000000,2020.000000,1884.000000,1.000000,0.957544


In [179]:
review_df.isnull().sum()

Unnamed: 0              0
reviewerName            1
overall                 0
reviewText              1
reviewTime              0
day_diff                0
helpful_yes             0
helpful_no              0
total_vote              0
score_pos_neg_diff      0
score_average_rating    0
wilson_lower_bound      0
dtype: int64

## Step 1. Removing null values from dataframe

In [180]:
review_df.shape

(4915, 12)

In [181]:
review_df.dropna(inplace=True)

In [182]:
review_df.shape

(4913, 12)

## Step 2. Remove Unwanted features

In [183]:
review_df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
review_df = review_df.sort_values(ascending=False,by='wilson_lower_bound')
review_df = review_df.reset_index(drop=True)
review_df.head(4)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,"Hyoun Kim ""Faluzure""",5.0,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,2013-01-05,702,1952,68,2020,1884,0.966337,0.957544
1,NLee the Engineer,5.0,I have tested dozens of SDHC and micro-SDHC ca...,2012-09-26,803,1428,77,1505,1351,0.948837,0.936519
2,SkincareCEO,1.0,NOTE: please read the last update (scroll to ...,2013-05-08,579,1568,126,1694,1442,0.925620,0.912139
3,"Amazon Customer ""Kelly""",1.0,"If your card gets hot enough to be painful, it...",2012-02-09,1033,422,73,495,349,0.852525,0.818577


In [184]:
review_df['overall'].value_counts()

overall
5.0    3921
4.0     526
1.0     244
3.0     142
2.0      80
Name: count, dtype: int64

In [185]:
review_df_final = review_df[['overall','reviewText','wilson_lower_bound']]
review_df_final.head(3)

,overall,reviewText,wilson_lower_bound
0,5.0,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,0.957544
1,5.0,I have tested dozens of SDHC and micro-SDHC ca...,0.936519
2,1.0,NOTE: please read the last update (scroll to ...,0.912139


In [186]:
# python -m spacy download en_core_web_sm


# Explanation of below Pre Process Function:
    * Stop Words: Words like "is", "for", etc., will be removed since they are in the stopword list.
    * Punctuation: Tokens like "#", "@", ".", "<", ">" are removed since they are punctuation.
    * Digits: Tokens like "99" and "34working" are removed because they contain numbers.
    * Case-insensitive Uniqueness: The function ensures words like "Bye" and "bye" are considered the same and are only kept once.
    * Lemmatization: The words are lemmatized to their base form to make sure variations like "working" and "work" are treated consistently.

In [187]:
import string
import nltk
import spacy


# Load SpaCy model (use 'en_core_web_sm' for English)
nlp = spacy.load('en_core_web_sm')
stop_words = nltk.corpus.stopwords.words('english')

def pre_process(txt):
    tokens = nlp(txt)
    existed_tokens =set()
    filtered_tokens =[]

    # Filter out:
    # 1. Punctuation
    # 2. Digits or alphanumeric tokens
    # 3. Tokens containing non-alphabetic characters (e.g., Hello@, 34working)
    # 4. Ensure case-insensitive uniqueness based on lemmatization
    # 4. Stop words

    for token in tokens:
        # Lemmatize the token and convert to lowercase for uniformity
        lemmatized_word = token.lemma_.lower()
        if lemmatized_word not in existed_tokens and \
                token.text not in string.punctuation  and \
                not token.is_digit and \
                token.text.isalpha() and \
                token.text.lower() not in stop_words:
            
            filtered_tokens.append(token.text)
            existed_tokens.add(lemmatized_word) # Track lemmatized word to ensure uniqueness
    
    return filtered_tokens

In [188]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [189]:
pre_process("If # everything Hello@ everyone . Bye 99 is 34working correctly, < this testing in hospital for patient >  bye> EveryOne")

['everything',
 'everyone',
 'Bye',
 'correctly',
 'testing',
 'hospital',
 'patient']

## Now lets apply above Pre Process function on Reviews Dataset

In [190]:
review_df_final.dtypes

overall               float64
reviewText             object
wilson_lower_bound    float64
dtype: object

In [192]:
review_df_final['reviewText'].apply(pre_process)

0       [UPDATE, lovely, wife, bought, Samsung, Galaxy...
1       [tested, dozens, SDHC, micro, cards, One, dist...
2       [NOTE, please, read, last, update, scroll, bot...
3       [card, gets, hot, enough, painful, defective, ...
4       [Sandisk, announcement, first, GB, micro, SD, ...
                              ...                        
4908    [probably, Nth, memory, card, bought, sandisk,...
4909    [use, Samsung, Galaxy, store, photos, videos, ...
4910    [awesome, Worked, well, Surface, tablet, later...
4911    [amazing, amount, storage, putting, micro, chi...
4912    [heard, bad, things, gb, Micro, SD, card, crap...
Name: reviewText, Length: 4913, dtype: object

In [ ]:
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

# Load SpaCy model (use 'en_core_web_sm' for English)
nlp = spacy.load('en_core_web_sm')

def pre_process_batch(texts):
    # Initialize set to track unique lemmatized words (case-insensitive)
    existed_tokens = set()
    filtered_tokens_list = []
    
    for doc in nlp.pipe(texts, batch_size=1000, n_process=4):  # Use batch processing and parallelism
        #  nlp.pipe() Avoid the overhead of repeatedly calling nlp() for each sentence or document. 
        #  This can speed up processing dramatically for large volumes of text.
        filtered_tokens = []
        
        for token in doc:
            # Lemmatize the token and convert to lowercase for uniformity
            lemmatized_word = token.lemma_.lower()

            # Filter out punctuation, digits, stop words, and ensure uniqueness based on lemmatized form
            if lemmatized_word not in existed_tokens and \
               token.text not in string.punctuation and \
               not token.isdigit() and \
               token.text.isalpha() and \
               token.text.lower() not in STOP_WORDS:
                filtered_tokens.append(token.text)
                existed_tokens.add(lemmatized_word)  # Track lemmatized word to ensure uniqueness
        
        filtered_tokens_list.append(filtered_tokens)
    
    return filtered_tokens_list
